In [1]:
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt 
from urllib import request
import csv
import pandas as pd

In [2]:
data_url="https://raw.githubusercontent.com/CSC-CONICET/CURSO-LEARNING1/main/datasets/molinos.csv" 
fdata = request.urlopen(data_url)

In [3]:
header = np.genfromtxt(data_url, dtype= str, delimiter=',', max_rows = 1)
dataset = np.genfromtxt(data_url, dtype= float, delimiter=',', skip_header = 1)

In [4]:
pd.read_csv(fdata)

,P_1,P_2,P_3,P_4,P_5,P_6,P_7,P_8,P_9,P_10,...,P_36,P_37,P_38,P_39,P_40,P_41,P_42,P_43,Velocity,Direction
0,1324.1,1324.3,1330.0,1337.8,1338.1,1328.2,1229.8,1325.6,1256.1,1338.2,...,1296.6,1114.5,1238.1,1316.0,1299.9,1257.5,1138.4,1172.8,12.1,260.0
1,1497.4,1499.0,1499.1,1485.8,1435.6,1338.4,1273.5,1035.2,594.9,1495.0,...,1472.1,1395.8,1289.7,1416.0,1249.6,1114.5,802.1,679.2,10.8,254.0
2,1724.9,1750.3,1467.2,1317.3,1285.5,1178.1,989.0,798.8,376.6,1388.3,...,1370.0,1090.9,962.2,1148.0,1055.2,772.5,555.4,316.3,8.4,253.0
3,1594.7,1686.5,1394.2,1327.2,1247.6,1236.1,1170.3,889.0,255.9,1555.4,...,1185.4,1053.1,773.2,924.2,799.5,576.3,448.1,418.8,7.8,251.0
4,1645.4,1678.3,1537.7,1397.1,1403.9,1390.1,1278.6,654.6,181.3,1624.8,...,1335.1,1504.6,1014.5,1207.9,989.8,1030.7,906.8,816.2,7.8,248.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128545,358.2,585.9,122.1,78.2,89.4,265.9,444.2,418.8,437.5,509.8,...,437.1,382.6,456.1,415.7,389.4,335.4,370.4,416.3,5.5,355.0
128546,205.3,371.8,61.7,30.2,63.6,149.4,275.3,233.9,252.0,294.7,...,300.2,245.2,301.8,247.9,250.9,231.2,237.6,288.1,5.4,354.0
128547,230.3,375.3,73.4,51.0,65.5,128.3,220.1,191.4,184.8,311.9,...,324.2,257.8,317.6,287.4,279.9,250.1,265.2,291.1,4.5,355.0
128548,160.9,326.9,67.6,23.1,75.8,154.9,218.0,177.2,191.3,244.2,...,310.3,253.3,314.0,261.6,248.6,211.1,212.3,234.6,4.0,353.0


In [5]:
#separo datos en un 60% de entrenamiento, 40% validacion
train_limit = int(np.shape(dataset)[0]*0.6)

train_data = dataset[:train_limit]
validation_data = dataset[train_limit:]

potencias = train_data[:,:43]
velocidad = train_data[:,43]
direccion = train_data[:,44]
direccion = -direccion

#curado de datos
mask = np.where(potencias < 0)
potencias = np.delete(potencias, mask[0], axis = 0)
velocidad = np.delete(velocidad, mask[0])
direccion = np.delete(direccion, mask[0])

#expreso velocidad en ejes cartesianos
x_axis = velocidad*np.cos((direccion+270)*np.pi/180)
y_axis = velocidad*np.sin((direccion+270)*np.pi/180)
velocidad_cartesiana = np.hstack((x_axis,y_axis))
velocidad_cartesiana = velocidad_cartesiana.reshape(np.size(velocidad),2)

In [8]:
#normalizo datos de potencia
media = np.mean(potencias, axis=0)
desvio = np.std(potencias, axis=0)
potencias = (potencias-media)/desvio

In [8]:
#creacion del regresor
largo, largo_dim = potencias.shape
capas_ocultas = np.array([512, 256, 128, 64])

model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(capas_ocultas[0], input_dim=largo_dim))
model.add(tf.keras.layers.Dense(capas_ocultas[1], activation='leaky_relu',kernel_initializer='glorot_normal', input_dim=capas_ocultas[0]))
model.add(tf.keras.layers.Dropout(0.2))#agrego ruido
model.add(tf.keras.layers.Dense(capas_ocultas[2], activation='leaky_relu',kernel_initializer='he_normal',input_dim=capas_ocultas[1]))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(capas_ocultas[3], activation='leaky_relu',kernel_initializer='he_normal',input_dim=capas_ocultas[2]))
model.add(tf.keras.layers.Dense(2, input_dim= capas_ocultas[3]))
model.summary()

es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='MeanSquaredError')
hist = model.fit(x= potencias, y= velocidad_cartesiana, batch_size =largo, epochs=10000, callbacks=[es])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               22528     
                                                                 
 dense_11 (Dense)            (None, 256)               131328    
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_12 (Dense)            (None, 128)               32896     
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dense_14 (Dense)            (None, 2)                

In [9]:
#guardo modelos y pesos 
from keras.models import model_from_json
model_json = model.to_json()
model_json = model.to_json()
with open("molinos_model.json", "w") as json_file:
  json_file.write(model_json)
#serializan los pesos (weights) para HDF5
model.save_weights("molinos_model_weights.h5")
print("Modelo guardado en el PC")

Modelo guardado en el PC


In [10]:
#guardo datos de validacion
df = pd.DataFrame(validation_data, columns = header)
df.to_csv('test_data.csv', index=False)

In [11]:
df

,P_1,P_2,P_3,P_4,P_5,P_6,P_7,P_8,P_9,P_10,...,P_36,P_37,P_38,P_39,P_40,P_41,P_42,P_43,Velocity,Direction
0,190.5,233.1,182.0,168.5,165.9,142.2,163.6,131.6,87.0,145.3,...,158.6,153.6,121.4,103.7,112.5,129.3,98.1,121.3,4.5,285.0
1,185.2,161.4,193.8,174.4,195.2,143.8,125.1,125.7,113.5,206.3,...,102.8,77.6,93.5,78.6,37.5,46.4,77.4,74.6,4.3,283.0
2,159.2,113.7,124.5,107.1,114.3,124.3,89.8,93.5,116.2,105.0,...,72.3,97.1,45.7,59.8,24.3,36.8,46.3,62.7,4.6,272.0
3,111.2,140.0,113.6,125.2,111.4,75.9,102.4,111.9,119.0,98.5,...,54.3,83.1,64.9,33.5,53.9,51.4,51.1,70.6,4.3,268.0
4,128.0,105.3,94.8,126.2,132.0,93.2,97.3,92.5,92.5,101.0,...,48.9,40.7,28.4,48.4,75.5,53.7,32.8,-2.0,4.0,261.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51415,358.2,585.9,122.1,78.2,89.4,265.9,444.2,418.8,437.5,509.8,...,437.1,382.6,456.1,415.7,389.4,335.4,370.4,416.3,5.5,355.0
51416,205.3,371.8,61.7,30.2,63.6,149.4,275.3,233.9,252.0,294.7,...,300.2,245.2,301.8,247.9,250.9,231.2,237.6,288.1,5.4,354.0
51417,230.3,375.3,73.4,51.0,65.5,128.3,220.1,191.4,184.8,311.9,...,324.2,257.8,317.6,287.4,279.9,250.1,265.2,291.1,4.5,355.0
51418,160.9,326.9,67.6,23.1,75.8,154.9,218.0,177.2,191.3,244.2,...,310.3,253.3,314.0,261.6,248.6,211.1,212.3,234.6,4.0,353.0


In [25]:
#creo el script 

import tensorflow as tf 
import numpy as np
import pandas as pd
from keras.models import model_from_json

data_csv = pd.read_csv('/content/test_data.csv')
dataset = pd.DataFrame.to_numpy(data_csv)

json_file = open('/content/molinos_model.json', 'r')
loaded_model_json = json_file.read() 
json_file.close()

loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('/content/molinos_model_weights.h5')
loaded_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='MeanSquaredError')

def curado(dataset):
  potencias = dataset[:,:43]
  velocidad = dataset[:,43]
  direccion = dataset[:,44]
  direccion = -direccion

  mask = np.where(potencias < 0)
  potencias = np.delete(potencias, mask[0], axis = 0)
  velocidad = np.delete(velocidad, mask[0])
  direccion = np.delete(direccion, mask[0])

  x_axis = velocidad*np.cos((direccion+270)*np.pi/180)
  y_axis = velocidad*np.sin((direccion+270)*np.pi/180)
  velocidad_cartesiana = np.hstack((x_axis,y_axis))
  velocidad_cartesiana = velocidad_cartesiana.reshape(np.size(velocidad),2)

  media = np.mean(potencias, axis=0)
  desvio = np.std(potencias, axis=0)
  potencias = (potencias-media)/desvio

  return potencias, velocidad_cartesiana

powers, velocities = curado(dataset)

ecm = loaded_model.evaluate(powers, velocities)

print("Error cuadratico medio = ", ecm)


999/999 [==============================] - 2s 2ms/step - loss: 38.5086
Error cuadratico medio =  38.50861358642578
